# Practica 1

## Parte B

### Ejercicio 1:

In [1]:
pip install scikit-surprise

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.4/154.4 kB 3.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.4-cp311-cp311-linux_x86_64.whl size=2505181 sha256=7c07525fb914cc3fd44b338a917589b9643d44ac4c4439aff05c0cb6c3aa39d8
  Stored in directory: /root/.cache/pip/wheels/2a/8f/6e/7e2899163e2d85d8266daab4aa1cdabec7a6c56f83c015b5af
Successfully built scikit-surprise


In [2]:
import os
import pandas as pd
import numpy as np

# Variable SEED, para que cada vez que se ejecute el código salgan los mismos resultados
SEED = 123

In [3]:
# Cargar movielens-100k dataset
from surprise import Dataset
from surprise.model_selection import cross_validate

data = Dataset.load_builtin("ml-100k")

Dataset ml-100k could not be found. Do you want to download it? [Y/n] y
Trying to download dataset from https://files.grouplens.org/datasets/movielens/ml-100k.zip...
Done! Dataset ml-100k has been saved to /root/.surprise_data/ml-100k


### Ejercicio 2:

In [4]:
# Dividir data en particiones de entrenamiento (75%) y evaluacion (25%)
from surprise.model_selection import train_test_split

trainset, testset = train_test_split(data, test_size=0.25, random_state=SEED)

### Ejercicio 3/4:

In [5]:
# Filtrado KNNBasic basado en usarios
from surprise import KNNBasic

knn_basic_user = KNNBasic(sim_options={"name": "pearson", "user_based": True})
knn_basic_user.fit(trainset)

# Mostrar los resultados
cross_validate(knn_basic_user, data, measures=["RMSE", "MSE", "MAE"], cv=5, verbose=True)

Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Evaluating RMSE, MSE, MAE of algorithm KNNBasic on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    1.0106  1.0151  1.0083  1.0102  1.0109  1.0110  0.0022  
MSE (testset)     1.0213  1.0304  1.0167  1.0206  1.0219  1.0222  0.0045  
MAE (testset)     0.8003  0.8076  0.8026  0.8015  0.8012  0.8026  0.0026  
Fit time          1.51    1.64    0.66    0.67    0.70    1.04    0.44    
Test time         9.77    3.98    3.91    2.94    2.87    4.70    2.58    


{'test_rmse': array([1.01057608, 1.0150694 , 1.00830727, 1.01024233, 1.01088512]),
 'test_mse': array([1.02126401, 1.03036588, 1.01668356, 1.02058956, 1.02188872]),
 'test_mae': array([0.80029854, 0.80756292, 0.80257472, 0.80148114, 0.80117825]),
 'fit_time': (1.5147805213928223,
  1.6443846225738525,
  0.662806510925293,
  0.6713314056396484,
  0.695671796798706),
 'test_time': (9.769641399383545,
  3.97843074798584,
  3.9143471717834473,
  2.9445738792419434,
  2.871009349822998)}

In [6]:
# Filtrado KNNBasic basado en productos
from surprise import KNNBasic

knn_basic_product = KNNBasic(sim_options={"name": "pearson", "user_based": False})
knn_basic_product.fit(trainset)

# Mostrar los resultados
cross_validate(knn_basic_product, data, measures=["RMSE", "MSE", "MAE"], cv=5, verbose=True)

Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Evaluating RMSE, MSE, MAE of algorithm KNNBasic on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    1.0370  1.0419  1.0419  1.0417  1.0387  1.0402  0.0020  
MSE (testset)     1.0754  1.0855  1.0855  1.0850  1.0790  1.0821  0.0042  
MAE (testset)     0.8298  0.8375  0.8346  0.8320  0.8333  0.8334  0.0026  
Fit time          1.05    1.52    1.03    1.06    1.03    1.14    0.19    
Test time         3.85    3.71    3.46    4.32    3.36    3.74    0.34    


{'test_rmse': array([1.03702177, 1.04187155, 1.04188592, 1.04165056, 1.03874046]),
 'test_mse': array([1.07541415, 1.08549633, 1.08552626, 1.08503589, 1.07898174]),
 'test_mae': array([0.82979434, 0.83751266, 0.83457619, 0.8319718 , 0.83328766]),
 'fit_time': (1.0484342575073242,
  1.5198309421539307,
  1.0330345630645752,
  1.058267593383789,
  1.0252439975738525),
 'test_time': (3.8541953563690186,
  3.708451986312866,
  3.464574098587036,
  4.32409143447876,
  3.3636341094970703)}

In [7]:
# Filtrado colaborativo SVD
from surprise import SVD

svd = SVD(random_state=SEED)
svd.fit(trainset)

# Mostrar los resultados
cross_validate(svd, data, measures=["RMSE", "MSE", "MAE"], cv=5, verbose=True)

Evaluating RMSE, MSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9474  0.9338  0.9384  0.9371  0.9307  0.9375  0.0056  
MSE (testset)     0.8976  0.8720  0.8806  0.8781  0.8663  0.8789  0.0106  
MAE (testset)     0.7453  0.7348  0.7397  0.7403  0.7318  0.7384  0.0047  
Fit time          3.03    1.36    1.37    1.40    1.37    1.71    0.66    
Test time         0.23    0.11    0.28    0.11    0.10    0.16    0.07    


{'test_rmse': array([0.94742089, 0.9338221 , 0.93840385, 0.93708928, 0.93072941]),
 'test_mse': array([0.89760633, 0.87202372, 0.88060178, 0.87813632, 0.86625724]),
 'test_mae': array([0.74531055, 0.73478543, 0.73971007, 0.74034658, 0.73175561]),
 'fit_time': (3.033684730529785,
  1.3644604682922363,
  1.3659896850585938,
  1.396454095840454,
  1.3693695068359375),
 'test_time': (0.22719359397888184,
  0.10839247703552246,
  0.27541661262512207,
  0.1074836254119873,
  0.1047370433807373)}

In [8]:
# Filtrado colaborativo NMF
from surprise import NMF

nmf = NMF(random_state=SEED)
nmf.fit(trainset)

# Mostrar los resultados
cross_validate(nmf, data, measures=["RMSE", "MSE", "MAE"], cv=5, verbose=True)

Evaluating RMSE, MSE, MAE of algorithm NMF on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9634  0.9595  0.9611  0.9601  0.9639  0.9616  0.0018  
MSE (testset)     0.9281  0.9207  0.9238  0.9218  0.9291  0.9247  0.0034  
MAE (testset)     0.7562  0.7539  0.7548  0.7557  0.7558  0.7553  0.0008  
Fit time          2.11    2.08    2.08    2.61    2.08    2.19    0.21    
Test time         0.22    0.09    0.30    0.09    0.22    0.18    0.08    


{'test_rmse': array([0.96336788, 0.95950819, 0.96112691, 0.96008283, 0.96391966]),
 'test_mse': array([0.92807768, 0.92065597, 0.92376494, 0.92175904, 0.92914111]),
 'test_mae': array([0.75624875, 0.75387592, 0.75480837, 0.75571971, 0.75575287]),
 'fit_time': (2.1080875396728516,
  2.0824878215789795,
  2.08467173576355,
  2.6063120365142822,
  2.0848093032836914),
 'test_time': (0.22012829780578613,
  0.0870053768157959,
  0.3027760982513428,
  0.08519148826599121,
  0.2230525016784668)}

### Ejercicio 5:

In [9]:
# Hacer predicciones para conjunto de evaluacion con cada algoritmo
from surprise import accuracy

predictions_user = knn_basic_user.test(testset)
predictions_product = knn_basic_product.test(testset)
predictions_svd = svd.test(testset)
predictions_nmf = nmf.test(testset)

# Mostrar 5 resultados de cada predicciones
N = 5

res_user = predictions_user[:N]
res_product = predictions_product[:N]
res_svd = predictions_svd[:N]
res_nmf = predictions_nmf[:N]

from pprint import pprint

print("FILTRADO BASADO EN USUARIOS")
pprint(res_user)
print()
print("FILTRADO BASADO EN PRODUCTOS")
pprint(res_product)
print()
print("FILTRADO COLLABORATIVO SVD")
pprint(res_svd)
print()
print("FILTRADO COLLABORATIVO NMF")
pprint(res_nmf)
print()

FILTRADO BASADO EN USUARIOS
[Prediction(uid='798', iid='480', r_ui=3.0, est=4.053348065550606, details={'actual_k': 40, 'was_impossible': False}),
 Prediction(uid='296', iid='462', r_ui=4.0, est=3.9480786343003986, details={'actual_k': 40, 'was_impossible': False}),
 Prediction(uid='653', iid='258', r_ui=3.0, est=3.3066313010563, details={'actual_k': 40, 'was_impossible': False}),
 Prediction(uid='883', iid='239', r_ui=3.0, est=3.193667587346957, details={'actual_k': 40, 'was_impossible': False}),
 Prediction(uid='272', iid='187', r_ui=5.0, est=4.576872850394524, details={'actual_k': 40, 'was_impossible': False})]

FILTRADO BASADO EN PRODUCTOS
[Prediction(uid='798', iid='480', r_ui=3.0, est=2.748698843329133, details={'actual_k': 40, 'was_impossible': False}),
 Prediction(uid='296', iid='462', r_ui=4.0, est=4.068686285129716, details={'actual_k': 40, 'was_impossible': False}),
 Prediction(uid='653', iid='258', r_ui=3.0, est=2.155357167971347, details={'actual_k': 40, 'was_impossible': 

Conclusiones:
Podemos ver que el atributo "est" (estimated rating) de los resultados en cada algoritmo cambia. En ninguno de los casos todas las valores de prediccion son bastante cercas a referencia (e.g. si r_ui = 3.0 y est = 4, no es bastance cerca).  

### Ejercicio 6:

In [10]:
# Crear tabla RMSE, precision@k, recall@k, NDCG, con k = 10 (solo peliculas con rating 4 o mas)
from surprise import accuracy

# Evaluacion RMSE
rmse_user = accuracy.rmse(predictions_user, verbose=True)
rmse_product = accuracy.rmse(predictions_product, verbose=True)
rmse_svd = accuracy.rmse(predictions_svd, verbose=True)
rmse_nmf = accuracy.rmse(predictions_nmf, verbose=True)

k = 10

RMSE: 0.8116
RMSE: 0.8265
RMSE: 0.7362
RMSE: 0.8251


In [44]:
# Evaluacion precision usando precision_score
from sklearn.metrics import precision_score

# predictions r_ui (real)
predictions_user_r = [pred.r_ui for pred in predictions_user]
predictions_product_r = [pred.r_ui for pred in predictions_product]
predictions_svd_r = [pred.r_ui for pred in predictions_svd]
predictions_nmf_r = [pred.r_ui for pred in predictions_nmf]

# predictions est
predictions_user_est = [pred.est for pred in predictions_user]
predictions_product_est = [pred.est for pred in predictions_product]
predictions_svd_est = [pred.est for pred in predictions_svd]
predictions_nmf_est = [pred.est for pred in predictions_nmf]

# transformar a valores discretos
from sklearn.preprocessing import KBinsDiscretizer

predictions_user_r_discrete = KBinsDiscretizer(n_bins=5, encode='ordinal', strategy='uniform').fit_transform(np.array(predictions_user_r).reshape(-1, 1))
predictions_product_r_discrete = KBinsDiscretizer(n_bins=5, encode='ordinal', strategy='uniform').fit_transform(np.array(predictions_product_r).reshape(-1, 1))
predictions_svd_r_discrete = KBinsDiscretizer(n_bins=5, encode='ordinal', strategy='uniform').fit_transform(np.array(predictions_svd_r).reshape(-1, 1))
predictions_nmf_r_discrete = KBinsDiscretizer(n_bins=5, encode='ordinal', strategy='uniform').fit_transform(np.array(predictions_nmf_r).reshape(-1, 1))

predictions_user_est_discrete = KBinsDiscretizer(n_bins=5, encode='ordinal', strategy='uniform').fit_transform(np.array(predictions_user_est).reshape(-1, 1))
predictions_product_est_discrete = KBinsDiscretizer(n_bins=5, encode='ordinal', strategy='uniform').fit_transform(np.array(predictions_product_est).reshape(-1, 1))
predictions_svd_est_discrete = KBinsDiscretizer(n_bins=5, encode='ordinal', strategy='uniform').fit_transform(np.array(predictions_svd_est).reshape(-1, 1))
predictions_nmf_est_discrete = KBinsDiscretizer(n_bins=5, encode='ordinal', strategy='uniform').fit_transform(np.array(predictions_nmf_est).reshape(-1, 1))

precision_user = precision_score(predictions_user_r_discrete, predictions_user_est_discrete, average='micro')
precision_product = precision_score(predictions_product_r_discrete, predictions_product_est_discrete, average='micro')
precision_svd = precision_score(predictions_svd_r_discrete, predictions_svd_est_discrete, average='micro')
precision_nmf = precision_score(predictions_nmf_r_discrete, predictions_nmf_est_discrete, average='micro')

print("FILTRADO BASADO EN USUARIOS")
print(precision_user)
print()
print("FILTRADO BASADO EN PRODUCTOS")
print(precision_product)
print()
print("FILTRADO COLLABORATIVO SVD")
print(precision_svd)
print()
print("FILTRADO COLLABORATIVO NMF")
print(precision_nmf)
print()

FILTRADO BASADO EN USUARIOS
0.49864

FILTRADO BASADO EN PRODUCTOS
0.4854

FILTRADO COLLABORATIVO SVD
0.55172

FILTRADO COLLABORATIVO NMF
0.4938



In [50]:
# evaluacion de recall usando recall_score
from sklearn.metrics import recall_score

recall_user = recall_score(predictions_user_r_discrete, predictions_user_est_discrete, average='micro')
recall_product = recall_score(predictions_product_r_discrete, predictions_product_est_discrete, average='micro')
recall_svd = recall_score(predictions_svd_r_discrete, predictions_svd_est_discrete, average='micro')
recall_nmf = recall_score(predictions_nmf_r_discrete, predictions_nmf_est_discrete, average='micro')

print("FILTRADO BASADO EN USUARIOS")
print(recall_user)
print()
print("FILTRADO BASADO EN PRODUCTOS")
print(recall_product)
print()
print("FILTRADO COLLABORATIVO SVD")
print(recall_svd)
print()
print("FILTRADO COLLABORATIVO NMF")
print(recall_nmf)
print()

FILTRADO BASADO EN USUARIOS
0.49864

FILTRADO BASADO EN PRODUCTOS
0.4854

FILTRADO COLLABORATIVO SVD
0.55172

FILTRADO COLLABORATIVO NMF
0.4938



In [52]:
import numpy as np
from sklearn.metrics import ndcg_score

# asegurar para que predicciones estan en un array 2D: n_samples, n_items
# cada query tiene varios items (relevantes e irrelevantes)

def reshape_for_ndcg(predictions_r, predictions_est, k):
    return np.array([predictions_r[:k]]).reshape(1, -1), np.array([predictions_est[:k]]).reshape(1, -1)

predictions_user_r_discrete, predictions_user_est_discrete = reshape_for_ndcg(predictions_user_r_discrete, predictions_user_est_discrete, k)
predictions_product_r_discrete, predictions_product_est_discrete = reshape_for_ndcg(predictions_product_r_discrete, predictions_product_est_discrete, k)
predictions_svd_r_discrete, predictions_svd_est_discrete = reshape_for_ndcg(predictions_svd_r_discrete, predictions_svd_est_discrete, k)
predictions_nmf_r_discrete, predictions_nmf_est_discrete = reshape_for_ndcg(predictions_nmf_r_discrete, predictions_nmf_est_discrete, k)

ndcg_user = ndcg_score(predictions_user_r_discrete, predictions_user_est_discrete, k=k)
ndcg_product = ndcg_score(predictions_product_r_discrete, predictions_product_est_discrete, k=k)
ndcg_svd = ndcg_score(predictions_svd_r_discrete, predictions_svd_est_discrete, k=k)
ndcg_nmf = ndcg_score(predictions_nmf_r_discrete, predictions_nmf_est_discrete, k=k)

print("FILTRADO BASADO EN USUARIOS")
print(ndcg_user)
print()
print("FILTRADO BASADO EN PRODUCTOS")
print(ndcg_product)
print()
print("FILTRADO COLLABORATIVO SVD")
print(ndcg_svd)
print()
print("FILTRADO COLLABORATIVO NMF")
print(ndcg_nmf)
print()

FILTRADO BASADO EN USUARIOS
0.9688524602256489

FILTRADO BASADO EN PRODUCTOS
0.9931895950259789

FILTRADO COLLABORATIVO SVD
0.9807221885991098

FILTRADO COLLABORATIVO NMF
0.9807221885991098



In [56]:
# crear tabla consolidada de RMSE, precision, recall, ndcg
import pandas as pd

data = {'Algoritmo': ['Basado en Usuarios', 'Basado en Productos', 'SVD', 'NMF'],
  'RMSE': [
    rmse_user,
    rmse_product,
    rmse_svd,
    rmse_nmf
  ],
  'Precision': [
    precision_user,
    precision_product,
    precision_svd,
    precision_nmf
  ],
  'Recall': [
    recall_user,
    recall_product,
    recall_svd,
    recall_nmf
  ],
  'NDCG': [
    ndcg_user,
    ndcg_product,
    ndcg_svd,
    ndcg_nmf
  ]
}

df = pd.DataFrame(data)
df

,Algoritmo,RMSE,Precision,Recall,NDCG
0,Basado en Usuarios,0.811566,0.49864,0.49864,0.968852
1,Basado en Productos,0.826469,0.48540,0.48540,0.993190
2,SVD,0.736210,0.55172,0.55172,0.980722
3,NMF,0.825097,0.49380,0.49380,0.980722


from matplotlib import pyplot as plt
df['RMSE'].plot(kind='hist', bins=20, title='RMSE')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
df['Precision'].plot(kind='hist', bins=20, title='Precision')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
df['Recall'].plot(kind='hist', bins=20, title='Recall')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
df['NDCG'].plot(kind='hist', bins=20, title='NDCG')
plt.gca().spines[['top', 'right',]].set_visible(False)

...

from matplotlib import pyplot as plt
import seaborn as sns
figsize = (12, 1.2 * len(df['Algoritmo'].unique()))
plt.figure(figsize=figsize)
sns.violinplot(df, x='Precision', y='Algoritmo', inner='stick', palette='Dark2')
sns.despine(top=True, right=True, bottom=True, left=True)

<string>:5: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `y` variable to `hue` and set `legend=False` for the same effect.



from matplotlib import pyplot as plt
import seaborn as sns
figsize = (12, 1.2 * len(df['Algoritmo'].unique()))
plt.figure(figsize=figsize)
sns.violinplot(df, x='Recall', y='Algoritmo', inner='stick', palette='Dark2')
sns.despine(top=True, right=True, bottom=True, left=True)

<string>:5: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `y` variable to `hue` and set `legend=False` for the same effect.



from matplotlib import pyplot as plt
import seaborn as sns
figsize = (12, 1.2 * len(df['Algoritmo'].unique()))
plt.figure(figsize=figsize)
sns.violinplot(df, x='NDCG', y='Algoritmo', inner='stick', palette='Dark2')
sns.despine(top=True, right=True, bottom=True, left=True)

Conclusiones: el algoritmo recomendado es SVD por su error RMSE comparativamente baja, y su precision y recall mas altos de todos.